11/15-17/23

More pruning for SPD call dataset for CSE 442 FP (Explorable Explanation). My team and I determined some call priorities and types that are irrelevant to our project, so I need to get rid of them.

In [31]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re

In [32]:
filename = 'call_data_pruned_2.csv'
ogData = pd.read_csv(filename)

In [33]:
ogData.shape[0]

2843445

In [34]:
data = ogData[(ogData["Priority"] < 5)] # determined that anything 5 and beyond wasn't a typical police call and was irrelevant to us
data = data[(data["Call Type"] != "IN PERSON COMPLAINT") & (data["Call Type"] != "HISTORY CALL (RETRO)") & (data["Call Type"] != "PROACTIVE (OFFICER INITIATED)") 
              & (data["Call Type"] !=  "ALARM CALL (NOT POLICE ALARM)") & (data["Call Type"] !=  "POLICE (VARDA) ALARM")] # irrelevant or rare general call types
data.reset_index(drop=True, inplace=True)

In [35]:
data.shape[0]

2509670

These next few cells get rid of call types that start with hyphens. These call types related to TRU (telephone reporting unit, we thought) and other irrelevant things. I took them out by making a new column that was just the first character of every call type, removing all rows with call types starting with hyphens, and then dropping that column.

In [36]:
data["temp"] = [string[0:1] for string in data["Initial Call Type"]]

In [37]:
print(data["temp"][0:5])

0    P
1    T
2    T
3    T
4    S
Name: temp, dtype: object


In [38]:
data = data[(data["temp"] != "-")]
data.reset_index(drop=True, inplace=True)
data = data.drop("temp", axis=1)

In [39]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2509591 entries, 0 to 2509590
Data columns (total 12 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Call Type          object 
 1   Priority           float64
 2   Initial Call Type  object 
 3   Precinct           object 
 4   Beat               object 
 5   Blurred_Longitude  float64
 6   Blurred_Latitude   float64
 7   Response Time      float64
 8   Neighborhood       object 
 9   Og_Date            object 
 10  Og_Year            int64  
 11  Og_Time            object 
dtypes: float64(4), int64(1), object(7)
memory usage: 229.8+ MB


In [40]:
#data.to_csv('call_data_pruned_pre_consol.csv', index=False)

In [79]:
data_abb = data.copy()

In [80]:
data_abb = data_abb[(data_abb["Og_Year"] != 2009)] # 2009's data wasn't complete (found via SQL)
data_abb.reset_index(drop=True, inplace=True)

In [81]:
print(np.where(np.array(data_abb["Neighborhood"].isna()) == True)) # found in SQL that some calls had weird beats
# meaning that some rows didn't have neighborhoods
# we wanted to get rid of those

(array([    696,    1759,    2774, ..., 2412150, 2412872, 2412906],
      dtype=int64),)


In [82]:
data_abb = data_abb[(data_abb["Neighborhood"].isna() == False)]
data_abb.reset_index(drop=True, inplace=True)

In [83]:
data_abb["temp2"] = [string[0:3] for string in data_abb["Initial Call Type"]]
data_abb = data_abb[(data_abb["temp2"] != "TRU")] #getting rid of more TRU call types
data_abb.reset_index(drop=True, inplace=True)
data_abb = data_abb.drop("temp2", axis=1)

In [84]:
data_abb.shape[0]

2402777

In [85]:
data_abb["is_IP_JO"] = [(("IP" in string or "JO" in string) and "INCLUDES IP/JO" not in string) for string in data_abb["Initial Call Type"]]
# made a column indicating whether calls related to in progress/just occurred crimes (this was never used)

In [86]:
temp = np.array(data_abb["is_IP_JO"])
len(np.nonzero(np.where(temp == True, 1, 0))[0])

283555

In [87]:
data_abb["is_DV"] = [("DV" in string) for string in data_abb["Initial Call Type"]]
# made a column indicating whether calls related to domestic violence crimes (this was never used)

In [88]:
temp = np.array(data_abb["is_DV"])
len(np.nonzero(np.where(temp == True, 1, 0))[0])

155052

In [89]:
# wanted to make call types more understandable and generalizable (e.g. aggregate "ASLT - IP/JO", "ASLT - DV", into just "Assault")
# however, this required changing a lot of types by hand, due to inconsistent conventions
# e.g. most call types have an abbreviation, which is what I want, then a hyphen
# but "SERVICE - WELFARE CHECK" has what I want after the hyphen
def find_first_space(calltype):
    if calltype == "AUTO THEFT-IP/JO - VEHICLE, PLATES, TABS":
        return "AUTO THEFT".title() 
    if calltype == "BOMB - THREATS":
        return "BOMB THREATS".title()
    if calltype == "DIST - DV - NO ASLT" or calltype == "DIST - IP/JO - DV DIST - NO ASLT":
        return "Disturbance (Domestic Violence)".title()
    if calltype == "DISTURBANCE, MISCELLANEOUS/OTHER":
        return "Disturbance (Miscellaneous/Other)".title()
    if calltype == "SUICIDE, SUICIDAL PERSON AND ATTEMPTS":
        return "SUICIDE".title()
    if calltype == "FIGHT - IP - PHYSICAL (NO WEAPONS)" or calltype == "FIGHT - JO - PHYSICAL (NO WEAPONS)" or calltype == "FIGHT - VERBAL/ORAL (NO WEAPONS)":
        return "FIGHT (NO WEAPONS)".title()
    if calltype == "FIGHT - IP/JO - WITH WEAPONS" or calltype == "FIGHT - WITH WEAPONS":
        return "FIGHT (WITH WEAPONS)".title()
    if calltype == "KNOWN KIDNAPPNG":
        return "KNOWN KIDNAPPING".title()
    if calltype == "PARKING VIOLATION (EXCEPT ABANDONED CAR)":
        return "PARKING VIOLATION".title()
    if calltype == "PARKS VIOLATIONS/CITES (INCLUDES EXCLUSIONS)":
        return "PARKS VIOLATIONS/CITES".title()
    if calltype == "SERVICE - WELFARE CHECK":
        return "WELFARE CHECK".title()
    if calltype == "ROBBERY (INCLUDES STRONG ARM)":
        return "ROBBERY".title()
    if calltype == "THREATS (INCLS IN-PERSON/BY PHONE/IN WRITING)":
        return "THREATS".title()
    if calltype == "PEACE-STANDBY TO ASSURE (NO COURT ORDR SVC)":
        return "Peace (Standby To Assure)"
    if calltype == "OPEN - BUILDING, DOOR, ETC.":
        return "Open Building, Door, Etc."
    if calltype == "ANIMAL, REPORT - BITE":
        return "Animal"
    if calltype == "WEAPN-IP/JO-GUN,DEADLY WPN (NO THRT/ASLT/DIST)":
        return "Weapon"
    index = calltype.find(" -")
    if index != -1:
        temp = calltype[:(index)].strip()
        if temp == "ASLT":
            return "Assault"
        if temp == "BURG":
            return "Burglary"
        if temp == "DOWN":
            return "Person Down"
        if temp == "Found":
            return "Person Found"
        if temp == "HARAS":
            return "Harassment"
        if temp == "HAZ":
            return "Hazard"
        if temp == "HZMAT":
            return "Hazmat"
        if temp == "PHONE":
            return "Phone Calls"
        if temp == "WEAPN":
            return "Weapon"
        if temp == "ACC":
            return "Accident"
        if temp == "MVC":
            return "Motor Vehicle Crash"
        if temp == "AWOL" or temp == "DOA" or temp == "DUI" or temp == "NORAD" or temp == "SWAT":
            return temp
        return temp.title() # TRAFFIC - ASSIST MOTORIST
    return calltype.title() # PERSON IN BEHAVIORAL/EMOTIONAL CRISIS

In [90]:
data_abb["Abbrev_Type"] = [find_first_space(string) for string in data_abb["Initial Call Type"]]

In [91]:
print(data_abb["Abbrev_Type"][:10])

0                               Parking Violation
1                                         Threats
2                                         Threats
3                                         Threats
4                              Suspicious Package
5                                        Burglary
6                                           Noise
7    Trees Down, Obstructing Public Prop (No Haz)
8                                       Narcotics
9                                        Property
Name: Abbrev_Type, dtype: object


In [92]:
data_abb.head(5)

,Call Type,Priority,Initial Call Type,Precinct,Beat,Blurred_Longitude,Blurred_Latitude,Response Time,Neighborhood,Og_Date,Og_Year,Og_Time,is_IP_JO,is_DV,Abbrev_Type
0,"TELEPHONE OTHER, NOT 911",4.0,PARKING VIOLATION (EXCEPT ABANDONED CAR),SOUTHWEST,W3,-122.373896,47.536470,52.600000,Fauntleroy SW,2022-12-28,2022,07:54:06,False,False,Parking Violation
1,911,2.0,THREATS - DV - NO ASSAULT,NORTH,L1,-122.293701,47.729170,8.633333,Lake City,2016-07-24,2016,21:14:25,False,True,Threats
2,"TELEPHONE OTHER, NOT 911",2.0,THREATS - DV - NO ASSAULT,WEST,D3,-122.338465,47.620874,137.650000,SLU/Cascade,2019-09-24,2019,18:01:17,False,True,Threats
3,911,2.0,THREATS - DV - NO ASSAULT,WEST,D3,-122.333634,47.621987,22.000000,SLU/Cascade,2017-07-29,2017,17:38:30,False,True,Threats
4,911,1.0,SUSPICIOUS PACKAGE,WEST,D2,-122.342323,47.627573,23.083333,Queen Anne,2016-03-02,2016,00:18:39,False,False,Suspicious Package


In [93]:
data_abb = data_abb.drop("Initial Call Type", axis=1) # don't need original column anymore because it was converted

In [94]:
arr_remove = np.array(['AMBER'.title(),
'ASSIST OTHER AGENCY'.title(),
'ASSIST PUBLIC'.title(),
'ASSIST SPD'.title(),
'FOLLOW UP'.title(),
'HELP THE OFFICER'.title(),
'HOSPITAL GUARD ASSIGNMENT'.title(),
'INFORMATIONAL BROADCASTS'.title(),
'NARCOTICS WARRANT SERVICE'.title(),
'ORDER'.title(),
'PREMISE CHECK, OFFICER INITIATED ONVIEW ONLY'.title(),
'PREPLANNED EVENT'.title(),
'REQUEST TO WATCH'.title(),
'SUSPICIOUS STOP'.title(),
'SAMPLE / TEST CALL ONLY'.title(),
'SFD'.title(),
'STADIUM EVENT ASSIGNMENT'.title(),
'TEST CALL ONLY'.title(),
'WARRANT'.title(),
'WARRANT PICKUP'.title()]) # more irrelevant call types

In [95]:
for k in range(len(arr_remove)): # remove all call types we don't want
    #print(arr_remove[k])
    data_abb = data_abb[(data_abb['Abbrev_Type'] != arr_remove[k])]
    data_abb.reset_index(drop=True, inplace=True)

In [98]:
data_abb.reset_index(drop=True, inplace=True)
data_abb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2278401 entries, 0 to 2278400
Data columns (total 14 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Call Type          object 
 1   Priority           float64
 2   Precinct           object 
 3   Beat               object 
 4   Blurred_Longitude  float64
 5   Blurred_Latitude   float64
 6   Response Time      float64
 7   Neighborhood       object 
 8   Og_Date            object 
 9   Og_Year            int64  
 10  Og_Time            object 
 11  is_IP_JO           bool   
 12  is_DV              bool   
 13  Abbrev_Type        object 
dtypes: bool(2), float64(4), int64(1), object(7)
memory usage: 212.9+ MB


In [99]:
#data_abb.to_csv('call_data_pruned_final.csv', index=False) #old file name
# file name just for call types